In [221]:
import pandas as pd
import numpy as np
import spotipy
from spotipy.oauth2 import SpotifyOAuth
import regex

from urllib.parse import urlsplit, parse_qs

from sklearn.preprocessing import LabelEncoder, MinMaxScaler

from song_orderer import Playlist

import cred

pd.set_option('display.max_columns', None)

# Redesign

In [222]:
Playlist = Playlist('https://open.spotify.com/playlist/5s3Uj6XMk3ytXHDajpdKE2?si=b71f01ef873341de')

In [223]:
playlist = Playlist.playlist_metadata_df[['Track ID', 'Track Name', 'Artist Name', 'Danceability', 'Energy', 'Popularity', 'Loudness', 'Valence', 'Tempo']]

In [224]:
playlist.describe()

,Danceability,Energy,Popularity,Loudness,Valence,Tempo
count,63.000000,63.000000,63.000000,63.000000,63.000000,63.000000
mean,0.713952,0.848222,39.222222,-6.604857,0.524962,140.532476
std,0.085826,0.127002,16.173898,2.054647,0.260521,15.105814
min,0.443000,0.571000,6.000000,-15.215000,0.034700,77.508000
25%,0.666000,0.741000,27.000000,-7.617500,0.364500,132.001000
50%,0.723000,0.889000,37.000000,-6.404000,0.533000,142.960000
75%,0.764000,0.954000,50.500000,-5.244500,0.732500,148.499000
max,0.891000,0.999000,78.000000,-2.802000,0.972000,174.011000


In [225]:
# Account for half time tempos
playlist['Tempo'] = playlist['Tempo'].apply(lambda x: 2*x if x<100 else x)

/var/folders/kr/b0myfb8n375_bzh05rj27xjm0000gn/T/ipykernel_94495/1009048554.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  playlist['Tempo'] = playlist['Tempo'].apply(lambda x: 2*x if x<100 else x)


In [226]:
playlist['Loudness'] = np.abs(playlist['Loudness'])

/var/folders/kr/b0myfb8n375_bzh05rj27xjm0000gn/T/ipykernel_94495/2665594656.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  playlist['Loudness'] = np.abs(playlist['Loudness'])


In [227]:
# Normalise the columns
scaler = MinMaxScaler()
for column in playlist.drop(columns=['Track ID', 'Track Name', 'Artist Name']).columns:
    playlist[column] = scaler.fit_transform(playlist[[column]])

/var/folders/kr/b0myfb8n375_bzh05rj27xjm0000gn/T/ipykernel_94495/2935010741.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  playlist[column] = scaler.fit_transform(playlist[[column]])
/var/folders/kr/b0myfb8n375_bzh05rj27xjm0000gn/T/ipykernel_94495/2935010741.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  playlist[column] = scaler.fit_transform(playlist[[column]])
/var/folders/kr/b0myfb8n375_bzh05rj27xjm0000gn/T/ipykernel_94495/2935010741.py:4: SettingWithCopyWarning: 
A value is trying to be set o

In [228]:
# Map 1 to 0 and 0 to 0
playlist['Loudness'] = playlist['Loudness'].map(lambda x: 1 - x)
playlist['Inverse Popularity'] = playlist['Popularity'].map(lambda x: 1 - x)

/var/folders/kr/b0myfb8n375_bzh05rj27xjm0000gn/T/ipykernel_94495/238200573.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  playlist['Loudness'] = playlist['Loudness'].map(lambda x: 1 - x)
/var/folders/kr/b0myfb8n375_bzh05rj27xjm0000gn/T/ipykernel_94495/238200573.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  playlist['Inverse Popularity'] = playlist['Popularity'].map(lambda x: 1 - x)


In [229]:
playlist.describe()

,Danceability,Energy,Popularity,Loudness,Valence,Tempo,Inverse Popularity
count,63.000000,63.000000,63.000000,6.300000e+01,63.000000,63.000000,6.300000e+01
mean,0.604804,0.647715,0.461420,6.936392e-01,0.523058,0.309873,5.385802e-01
std,0.191577,0.296735,0.224637,1.655238e-01,0.277949,0.202087,2.246375e-01
min,0.000000,0.000000,0.000000,1.110223e-16,0.000000,0.000000,1.110223e-16
25%,0.497768,0.397196,0.291667,6.120599e-01,0.351862,0.152677,3.819444e-01
50%,0.625000,0.742991,0.430556,7.098203e-01,0.531633,0.306289,5.694444e-01
75%,0.716518,0.894860,0.618056,8.032305e-01,0.744479,0.418924,7.083333e-01
max,1.000000,1.000000,1.000000,1.000000e+00,1.000000,1.000000,1.000000e+00


In [230]:
playlist

,Track ID,Track Name,Artist Name,Danceability,Energy,Popularity,Loudness,Valence,Tempo,Inverse Popularity
0,3hzelPptWzx0OrSDTyIuuy,Flare,Hensonn,0.625000,0.831776,0.861111,0.978732,0.000000,0.419384,0.138889
1,4swEoEh5LpYOTDJHD5SL2w,Floor Seats,Dagga,0.388393,0.927570,0.097222,0.823975,0.262776,0.257523,0.902778
2,3pqvIzV7DIS9orbtKlHAal,Muschi Muschi,DBBD,0.703125,0.626168,0.597222,0.723355,0.880508,0.419448,0.402778
3,6igMEGFeVT03ZMrOEMzgyg,Fuck the Pa1n Away,DJ これからの緊急災害,0.497768,0.892523,0.333333,0.634738,0.743945,0.337784,0.666667
4,6GoLARmR2OZl2EldehFrsA,Immaterial,SOPHIE,0.743304,0.341121,0.000000,0.831225,0.846367,0.257555,1.000000
...,...,...,...,...,...,...,...,...,...,...
58,2tseEPCbD2xdVl86U4ZYgX,Talk To Me,Two Shell,0.649554,0.719626,0.736111,0.679046,0.758882,0.676746,0.263889
59,7C0PzxbsQx59DgGVRsquwJ,Voodoo People - Pendulum Mix,The Prodigy,0.000000,0.908879,0.652778,0.784178,0.531633,0.806112,0.347222
60,0fAuuR5hoWZtTMpuqqxAGy,home,Two Shell,0.642857,0.862150,0.486111,0.691291,0.679932,0.612692,0.513889
61,4FStZcPHUOx9Jj92fOpX1Q,H.O.O.D,KNEECAP,0.468750,0.637850,0.541667,0.839926,0.598848,0.726012,0.458333


In [231]:
# Create a composite metric to order the data frame by
composite_columns = [ 
    'Energy',
    'Danceability',
    'Loudness',
    #'Inverse Popularity',
    'Tempo',
    'Tempo',
    'Tempo'
]

# Add together the values of composite_columns in the playlist dataframe and divide by the number of columns
playlist['composite_metric'] = playlist[composite_columns].sum(axis=1)/len(composite_columns)

/var/folders/kr/b0myfb8n375_bzh05rj27xjm0000gn/T/ipykernel_94495/1841696591.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  playlist['composite_metric'] = playlist[composite_columns].sum(axis=1)/len(composite_columns)


In [232]:
# Order the data frame by the composite metric
ordered_playlist = playlist.sort_values(by=['composite_metric'], ascending=True)

In [233]:
ordered_playlist

,Track ID,Track Name,Artist Name,Danceability,Energy,Popularity,Loudness,Valence,Tempo,Inverse Popularity,composite_metric
24,2Zi6jLDMzAgfEN737xzx08,Touchpad,Two Shell,0.566964,0.193925,0.180556,1.110223e-16,0.080337,0.145025,0.819444,0.199327
21,1CUCnTJVZy68TS34DX6kCx,Sweat - SOPHIE Remix,SONIKKU,0.741071,0.044393,0.513889,4.342222e-01,0.429212,0.047444,0.486111,0.227003
22,42ixF3L8sfCJc9zu5DzmIl,Universe,Krister,0.482143,0.453271,0.555556,4.947233e-01,0.004268,0.015707,0.444444,0.246210
26,0jiThGoZ1qdJrImd4U6XE5,Good PusS (feat. cupcakKe) - Remix,COBRAH,0.897321,0.285047,0.569444,6.266817e-01,0.306519,0.015885,0.430556,0.309451
28,3CqaTHbiU2nBy3Ar9RnDHN,Baddy On The Floor,Jamie xx,0.466518,0.714953,0.541667,7.197293e-01,0.562573,0.000000,0.458333,0.316867
...,...,...,...,...,...,...,...,...,...,...,...
60,0fAuuR5hoWZtTMpuqqxAGy,home,Two Shell,0.642857,0.862150,0.486111,6.912914e-01,0.679932,0.612692,0.513889,0.672395
58,2tseEPCbD2xdVl86U4ZYgX,Talk To Me,Two Shell,0.649554,0.719626,0.736111,6.790462e-01,0.758882,0.676746,0.263889,0.679744
59,7C0PzxbsQx59DgGVRsquwJ,Voodoo People - Pendulum Mix,The Prodigy,0.000000,0.908879,0.652778,7.841779e-01,0.531633,0.806112,0.347222,0.685232
61,4FStZcPHUOx9Jj92fOpX1Q,H.O.O.D,KNEECAP,0.468750,0.637850,0.541667,8.399259e-01,0.598848,0.726012,0.458333,0.687427


In [234]:
peak_number = 20

high_songs = ordered_playlist.iloc[peak_number:len(ordered_playlist)]

high_songs_1 = high_songs[0:len(high_songs)//2]

high_songs_2 = high_songs[len(high_songs)//2:]

temp = ordered_playlist.drop(index=high_songs.index)
final_playlist = pd.concat([high_songs_1.sort_values(by='composite_metric', ascending=False), temp])
final_playlist = pd.concat([final_playlist, high_songs_2])

In [235]:
final_playlist

,Track ID,Track Name,Artist Name,Danceability,Energy,Popularity,Loudness,Valence,Tempo,Inverse Popularity,composite_metric
45,1eWRnjEjFVq6jT1KhApWgY,Groove Theory,X CLUB.,0.691964,0.843458,0.291667,0.710868,0.378001,0.322190,0.708333,0.535477
5,3jQQ3mGJ7ECgZDFVDLyK9h,That's the Funk Assault,Funk Assault,0.566964,0.927570,0.444444,0.670587,0.544436,0.321932,0.555556,0.521820
11,0gExQivfnTQxzD85envbyy,Arintintin,Rico Nasty,0.497768,0.651869,0.666667,0.862644,1.000000,0.370682,0.333333,0.520721
43,661kjkjwr1u6nUI6ek72v8,Lipstick Stain - Club Mix,SONIKKU,0.500000,0.750000,0.180556,0.570370,0.993599,0.419142,0.819444,0.512966
6,3zv6z0evv8QiIJjA8yOWAf,Pump This,EFESIAN,0.308036,1.000000,0.388889,0.850318,0.070735,0.305370,0.611111,0.512411
...,...,...,...,...,...,...,...,...,...,...,...
60,0fAuuR5hoWZtTMpuqqxAGy,home,Two Shell,0.642857,0.862150,0.486111,0.691291,0.679932,0.612692,0.513889,0.672395
58,2tseEPCbD2xdVl86U4ZYgX,Talk To Me,Two Shell,0.649554,0.719626,0.736111,0.679046,0.758882,0.676746,0.263889,0.679744
59,7C0PzxbsQx59DgGVRsquwJ,Voodoo People - Pendulum Mix,The Prodigy,0.000000,0.908879,0.652778,0.784178,0.531633,0.806112,0.347222,0.685232
61,4FStZcPHUOx9Jj92fOpX1Q,H.O.O.D,KNEECAP,0.468750,0.637850,0.541667,0.839926,0.598848,0.726012,0.458333,0.687427


In [236]:
final_playlist.reset_index(drop=True, inplace=True)

# Upload

In [237]:
items = final_playlist['Track ID'].tolist()

In [238]:
Playlist.update_playlist(items)